In [ ]:
import msal
print(msal.__version__)

## Graph

In [ ]:
from configparser import SectionProxy
from azure.identity import DeviceCodeCredential, ClientSecretCredential
from msgraph.core import GraphClient


In [ ]:
config = configparser.ConfigParser()
config.read(['config.cfg', 'config.dev.cfg'])
azure_settings = config['azure']

settings: SectionProxy
device_code_credential: DeviceCodeCredential
user_client: GraphClient
settings: SectionProxy

settings = azure_settings
# print(settings['authTenant'])
client_id = settings['clientId']
tenant_id = settings['authTenant']
graph_scopes = settings['graphUserScopes'].split(' ')

device_code_credential = DeviceCodeCredential(client_id, tenant_id = tenant_id)

user_client = GraphClient(credential=device_code_credential, scopes=graph_scopes)

graph_scopes = settings['graphUserScopes']

graph_scopes

access_token = device_code_credential.get_token(graph_scopes)


In [ ]:
access_token

In [ ]:
endpoint = '/me'
# Only request specific properties
select = 'displayName,mail,userPrincipalName'
request_url = f'{endpoint}?$select={select}'

user_response = user_client.get(request_url)
user_response.json()

In [ ]:
class Graph:
    
    settings: SectionProxy
    device_code_credential: DeviceCodeCredential
    user_client: GraphClient

    def __init__(self, config: SectionProxy):

        self.settings = config
        client_id = self.settings['clientId']
        tenant_id = self.settings['authTenant']
        graph_scopes = self.settings['graphUserScopes'].split(' ')

        self.device_code_credential = DeviceCodeCredential(client_id, tenant_id = tenant_id)

        self.user_client = GraphClient(credential=self.device_code_credential, scopes=graph_scopes)
    
    # <GetUserTokenSnippet>
    def get_user_token(self):
        graph_scopes = self.settings['graphUserScopes']
        access_token = self.device_code_credential.get_token(graph_scopes)
        return access_token.token

    def get_user(self):
        endpoint = '/me'
        # Only request specific properties
        select = 'displayName,mail,userPrincipalName'
        request_url = f'{endpoint}?$select={select}'

        user_response = self.user_client.get(request_url)
        return user_response.json()
    
    def get_inbox(self):
        endpoint = '/me/mailFolders/inbox/messages'
        # Only request specific properties
        select = 'from,isRead,receivedDateTime,subject'
        # Get at most 25 results
        top = 25
        # Sort by received time, newest first
        order_by = 'receivedDateTime DESC'
        request_url = f'{endpoint}?$select={select}&$top={top}&$orderBy={order_by}'

        inbox_response = self.user_client.get(request_url)
        return inbox_response.json()
        

# Task Specific function

In [ ]:
def greet_user(graph: Graph):
    user = graph.get_user()
    print('Hello,', user['displayName'])
    # For Work/school accounts, email is in mail property
    # Personal accounts, email is in userPrincipalName
    print('Email:', user['mail'] or user['userPrincipalName'], '\n')

In [ ]:
def display_access_token(graph: Graph):
    token = graph.get_user_token()
    print('User token:', token, '\n')

In [ ]:

def list_inbox(graph: Graph):
    message_page = graph.get_inbox()

    # Output each message's details
    for message in message_page['value']:
        print('Message:', message['subject'])
        print('  From:', message['from']['emailAddress']['name'])
        print('  Status:', 'Read' if message['isRead'] else 'Unread')
        print('  Received:', message['receivedDateTime'])

    # If @odata.nextLink is present
    more_available = '@odata.nextLink' in message_page
    print('\nMore messages available?', more_available, '\n')

In [ ]:

def send_mail(graph: Graph):
    # TODO
    return

In [ ]:

def list_users(graph: Graph):
    # TODO
    return

In [ ]:

def make_graph_call(graph: Graph):
    # TODO
    return

# Main python file

In [ ]:
import configparser
# from graph import Graph

In [ ]:
def main():
    print('Python Graph Tutorial\n')

    # Load settings
    config = configparser.ConfigParser()
    config.read(['config.cfg', 'config.dev.cfg'])
    azure_settings = config['azure']

    graph: Graph = Graph(azure_settings)

    greet_user(graph)

    choice = -1

    while choice != 0:
        print('Please choose one of the following options:')
        print('0. Exit')
        print('1. Display access token')
        print('2. List my inbox')
        print('3. Send mail')
        print('4. List users (requires app-only)')
        print('5. Make a Graph call')

        try:
            choice = int(input())
        except ValueError:
            choice = -1

        if choice == 0:
            print('Goodbye...')
        elif choice == 1:
            display_access_token(graph)
        elif choice == 2:
            list_inbox(graph)
        elif choice == 3:
            send_mail(graph)
        elif choice == 4:
            list_users(graph)
        elif choice == 5:
            make_graph_call(graph)
        else:
            print('Invalid choice!\n')

In [ ]:
main()